# Quest: Money Buddy

    - Goal: Classify ledger transactions
- Goal: Generate action plan schema with redaction

    **Pass criteria:** Action plan JSON created after categorisation.

    _Mock mode uses fixtures and writes artifacts under `_runs/`. Flip `USE_REAL_CLIENT=True` when ready for live calls._

In [ ]:

USE_REAL_CLIENT = False  # flip when ready
import os, json, time, pathlib, math, random

RUN = pathlib.Path('_runs/quest_money_buddy')
RUN.mkdir(parents=True, exist_ok=True)
FIXTURE_ROOT = pathlib.Path(os.environ.get('MOCK_FIXTURE_ROOT', '_fixtures'))

def write_meta(**kwargs):
    payload = {"timestamp": time.time(), **kwargs}
    (RUN / 'meta.json').write_text(json.dumps(payload, indent=2))
    return payload

USE_REAL_CLIENT = bool(str(os.environ.get('USE_REAL_CLIENT', USE_REAL_CLIENT)).lower() in {'1', 'true', 'yes'})


In [ ]:
import csv
from tools.redact_middleware import redact_emails
transactions = []
with (FIXTURE_ROOT / "transactions.csv").open() as handle:
    reader = csv.DictReader(handle)
    for row in reader:
        amount = float(row["amount"])
        category = "income" if amount > 0 else "expense"
        description = row["description"].lower()
        if "rent" in description:
            bucket = "housing"
        elif "gym" in description:
            bucket = "wellness"
        elif "coffee" in description:
            bucket = "lifestyle"
        elif "grocer" in description:
            bucket = "essentials"
        else:
            bucket = category
        transactions.append({**row, "amount": amount, "bucket": bucket})
plan_text = "Contact advisor at planner@example.com to review budget." if transactions else "No action needed."
redacted = redact_emails(plan_text)
plan = {
    "summary": {
        "income": sum(t["amount"] for t in transactions if t["amount"] > 0),
        "expenses": sum(-t["amount"] for t in transactions if t["amount"] < 0),
    },
    "steps": [
        {"name": "Review subscriptions", "impact": "medium"},
        {"name": "Set savings target", "impact": "high"},
    ],
    "notes": redacted.redacted,
    "redactions": redacted.replacements,
}
(RUN / "plan.json").write_text(json.dumps(plan, indent=2))
plan

In [ ]:
meta = write_meta(ok=True, lab="quest_money_buddy", redactions=plan['redactions'])
meta